% DES431 Project 2: Recommendation System

# Background

**MovieLens** is a movie recommendation system operated by GroupLens, a research group at the University of Minnesota. 

# Task

1. Propose and implement your own recommendation system based on the MovieLens dataset. Use `ratings_train.csv` as the training set, `ratings_valid.csv` as the validation set. Your system may use information from `movies.csv` and `tags.csv` to conduct recommendations. The undisclosed test set will be used to evaluate your system.
   - The data file structure is available at https://files.grouplens.org/datasets/movielens/ml-latest-small-README.html. 
   - The main goal of the recommendation system is to minimize the root-mean-square error.
   - The implementation should include a function named `predict_rating`. This function accepts a DataFrame with two columns `userId` and `movieId`. Then, the function adds a column named `rating` storing a predicted rating of a `movieId` by a `userId`.
   - Your program must return a root-mean-square error value when the validation set is changed to another file. Otherwise, your score will be deducted by 50%.
   - You must modify the given program to make better recommendations. Submitting the original program without modification is considered plagiarism.
2. Prepare slides for a 7-minute presentation to explain your proposed technique and algorithm to conduct recommendation, and show your RMSE results on the validation set.
3. Submit all required documents by April 30, 2023; 23:59. Late submission will not be accepted and will be marked 0. Do not wait until the last minute. Plagiarism and code duplication will be checked. 
4. Present your work on May 1, 2023 within 7 minutes. Exceeding 7 minutes will be subject to point deduction.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import imdb
import requests
import tmdbv3api as tmdb
from surprise import Dataset, Reader, SVD, accuracy

# Loading data

In [3]:
ratings_train = pd.read_csv('ratings_train.csv')
ratings_valid = pd.read_csv('ratings_valid.csv')
movies = pd.read_csv('movies.csv')
tags = pd.read_csv('tags.csv',usecols=["userId","movieId","tag"])
links = pd.read_csv('links.csv')

In [4]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [5]:
tags

,userId,movieId,tag
0,2,60756,funny
1,2,60756,Highly quotable
2,2,60756,will ferrell
3,2,89774,Boxing story
4,2,89774,MMA
...,...,...,...
3678,606,7382,for katie
3679,606,7936,austere
3680,610,3265,gun fu
3681,610,3265,heroic bloodshed


In [6]:
links

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [7]:
ratings_train

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
96459,610,166534,4.0,1493848402
96460,610,168248,5.0,1493850091
96461,610,168250,5.0,1494273047
96462,610,168252,5.0,1493846352


In [8]:
ratings_valid

,userId,movieId,rating,timestamp
0,4,45,3.0,986935047
1,4,52,3.0,964622786
2,4,58,3.0,964538444
3,4,222,1.0,945629040
4,4,247,3.0,986848894
...,...,...,...,...
2349,561,139385,3.5,1491092337
2350,561,146656,3.5,1491095479
2351,561,149406,3.5,1491091520
2352,561,160438,2.0,1491091498


In [9]:
print("Number of users = "+str(ratings_train["userId"].nunique()))
print("Number of movies = "+str(ratings_train["movieId"].nunique()))

Number of users = 610
Number of movies = 9690


# CONTENT-BASED

In [10]:
m_df=ratings_train[["userId","movieId","rating"]].merge(movies,on="movieId")
m_df

,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...,...,...
96459,610,160341,2.5,Bloodmoon (1997),Action|Thriller
96460,610,160527,4.5,Sympathy for the Underdog (1971),Action|Crime|Drama
96461,610,160836,3.0,Hazard (2005),Action|Drama|Thriller
96462,610,163937,3.5,Blair Witch (2016),Horror|Thriller


In [11]:
ratings = ratings_train.copy()

# convert UserID, MovieID to categorical type
ratings['userId'] = ratings['userId'].astype('category')
ratings['movieId'] = ratings['movieId'].astype('category')
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
96459,610,166534,4.0,1493848402
96460,610,168248,5.0,1493850091
96461,610,168250,5.0,1494273047
96462,610,168252,5.0,1493846352


In [12]:
# convert genres to Array
m_genre=m_df.copy()
m_genre["genres"]=m_genre["genres"].str.split("|")
# convert array elements to string
m_genre["genres"]=m_genre["genres"].fillna("").astype("str")
m_genre


,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story (1995),"['Adventure', 'Animation', 'Children', 'Comedy..."
1,5,1,4.0,Toy Story (1995),"['Adventure', 'Animation', 'Children', 'Comedy..."
2,7,1,4.5,Toy Story (1995),"['Adventure', 'Animation', 'Children', 'Comedy..."
3,15,1,2.5,Toy Story (1995),"['Adventure', 'Animation', 'Children', 'Comedy..."
4,17,1,4.5,Toy Story (1995),"['Adventure', 'Animation', 'Children', 'Comedy..."
...,...,...,...,...,...
96459,610,160341,2.5,Bloodmoon (1997),"['Action', 'Thriller']"
96460,610,160527,4.5,Sympathy for the Underdog (1971),"['Action', 'Crime', 'Drama']"
96461,610,160836,3.0,Hazard (2005),"['Action', 'Drama', 'Thriller']"
96462,610,163937,3.5,Blair Witch (2016),"['Horror', 'Thriller']"


In [13]:
# get movie title Series
titles = movies['title']
title_S = pd.Series(movies.index, index=movies['title'])
title_S

title
Toy Story (1995)                                0
Jumanji (1995)                                  1
Grumpier Old Men (1995)                         2
Waiting to Exhale (1995)                        3
Father of the Bride Part II (1995)              4
                                             ... 
Black Butler: Book of the Atlantic (2017)    9737
No Game No Life: Zero (2017)                 9738
Flint (2017)                                 9739
Bungo Stray Dogs: Dead Apple (2018)          9740
Andrew Dice Clay: Dice Rules (1991)          9741
Length: 9742, dtype: int64

In [14]:
# get genre name list
genre_name = set()
for genre in movies['genres'].str.split('|').values:
    genre_name = genre_name.union(set(genre))
genre_name

{'(no genres listed)',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western'}

# term-frequency inverse document frequency matrix

In [15]:
# find tf-idf score of genre for  each movie
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 1),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(m_genre['genres'])
print(tfidf_matrix[:10, :10].todense())

[[0.         0.36356033 0.549949   0.50819072 0.29282185 0.
  0.         0.         0.47050776 0.        ]
 [0.         0.36356033 0.549949   0.50819072 0.29282185 0.
  0.         0.         0.47050776 0.        ]
 [0.         0.36356033 0.549949   0.50819072 0.29282185 0.
  0.         0.         0.47050776 0.        ]
 [0.         0.36356033 0.549949   0.50819072 0.29282185 0.
  0.         0.         0.47050776 0.        ]
 [0.         0.36356033 0.549949   0.50819072 0.29282185 0.
  0.         0.         0.47050776 0.        ]
 [0.         0.36356033 0.549949   0.50819072 0.29282185 0.
  0.         0.         0.47050776 0.        ]
 [0.         0.36356033 0.549949   0.50819072 0.29282185 0.
  0.         0.         0.47050776 0.        ]
 [0.         0.36356033 0.549949   0.50819072 0.29282185 0.
  0.         0.         0.47050776 0.        ]
 [0.         0.36356033 0.549949   0.50819072 0.29282185 0.
  0.         0.         0.47050776 0.        ]
 [0.         0.36356033 0.549949   0.

In [16]:
# get movie title and rating from IMDB
linksIMDBID=links.copy()
linksIMDBID['imdbId'] = links['imdbId'].astype(str)

def get_movie_title_and_rating(links):
    def retrieve_info(imdbId):
        try:
            movie = ia.get_movie(imdbId)
            title = movie['title']
            rating = movie['rating']
            return title, rating
        except:
            return None, None
        
    # Apply the retrieve_info function to the imdbId column of the link_df
    movie_info = links['imdbId'].apply(retrieve_info)
    
    # Unpack the tuples into separate Series for title and rating
    movie_titles, movie_ratings = zip(*movie_info)
    
    # Return a DataFrame with the movie titles and ratings
    return pd.DataFrame({'movieId': links['movieId'], 'title': movie_titles, 'rating': movie_ratings})

# Apply the get_movie_title_and_rating function to the link DataFrame
movie_info_df = get_movie_title_and_rating(links)

In [17]:
# Get the indices of the movies in the tfidf_matrix
movie_indices = movies.index.tolist()

# Create the tfidf array only for the movies in the movies DataFrame
tfidf_array = tfidf_matrix[movie_indices].toarray()

#Create a new dupe for movies
movies_tf = movies.copy()
# Add the tfidf column to the movies DataFrame
movies_tf['Genre_tfidf'] = pd.Series(tfidf_array.tolist(), index=movies.index)
movies_tf


,movieId,title,genres,Genre_tfidf
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[0.0, 0.363560332296821, 0.5499489959184152, 0..."
1,2,Jumanji (1995),Adventure|Children|Fantasy,"[0.0, 0.363560332296821, 0.5499489959184152, 0..."
2,3,Grumpier Old Men (1995),Comedy|Romance,"[0.0, 0.363560332296821, 0.5499489959184152, 0..."
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,"[0.0, 0.363560332296821, 0.5499489959184152, 0..."
4,5,Father of the Bride Part II (1995),Comedy,"[0.0, 0.363560332296821, 0.5499489959184152, 0..."
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,"[0.44828009534257585, 0.0, 0.0, 0.0, 0.4001098..."
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,"[0.44828009534257585, 0.0, 0.0, 0.0, 0.4001098..."
9739,193585,Flint (2017),Drama,"[0.44828009534257585, 0.0, 0.0, 0.0, 0.4001098..."
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,"[0.44828009534257585, 0.0, 0.0, 0.0, 0.4001098..."


# Singular Value Decomposition (SVD)

In [18]:
from surprise import SVD
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy

# Set the rating scale
reader = Reader(rating_scale=(0, 5))

# Load the train dataset
train_data = Dataset.load_from_df(ratings_train[['userId', 'movieId', 'rating']], reader)

# Split the dataset into train and test sets
trainset, testset = train_test_split(train_data, test_size=0.2)

# Initialize the SVD model
model = SVD(n_factors=100, n_epochs=100, lr_all=0.005, reg_all=0.02)

# Train the model on the trainset
model.fit(trainset)

# Make predictions on the testset
predictions = model.test(testset)

# Compute and print the RMSE metric
rmse_SVD = accuracy.rmse(predictions)
print('RMSE:', rmse_SVD)

# Print the estimated ratings for each prediction
for prediction in predictions:
    print(prediction.est)


RMSE: 0.8823
RMSE: 0.8823126525902201
3.6722912419784897
3.9111259001573244
2.7679693244511263
3.092613878676525
4.381529105665109
2.8196926636854633
3.8933783493459186
3.598533826580692
5
3.661260040180639
4.59636188321491
3.6746841926436584
2.6048766477945615
3.3707495895369752
3.020161925168138
3.5878255718812677
4.176108681963271
3.02879701582727
3.3088877930472584
3.5512475231869636
2.667467702893332
3.3646172307941375
2.329397332171293
2.9680647665742343
2.799845105210566
3.8207489963244234
3.7534497463774654
3.290936873900254
3.9973149767130605
3.80791258614186
3.1409218632258824
3.9081262715956586
3.3474915329195483
3.6167699369998494
2.9323287608562034
4.201890831970009
2.354722397023297
4.6123910869940925
3.2454632282243647
2.8702068846567665
2.3442154680637066
2.7731889298079984
3.7088784128548222
3.584593718361533
3.349594130027518
3.2756685368416814
2.8582055553995267
3.2720605830830345
3.7131728046018417
3.1739420735890516
3.2005276744823394
3.4485697386407836
2.225193925

# Global Effect
# This one has lower RSME

In [27]:
from surprise import BaselineOnly
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy

# Set the rating scale
reader = Reader(rating_scale=(0, 5))

# Load the train dataset
train_data = Dataset.load_from_df(ratings_train[['userId', 'movieId', 'rating']], reader)

# Split the dataset into train and test sets
trainset, testset = train_test_split(train_data, test_size=0.2)

# Initialize the BaselineOnly model
model = BaselineOnly()

# Train the model on the trainset
model.fit(trainset)

# Make predictions on the testset
predictions_glob = model.test(testset)

# Compute and print the RMSE metric
rmse_globaleffect = accuracy.rmse(predictions)
print('RMSE:', rmse_globaleffect)

# Print the estimated ratings for each prediction
for prediction_glob in predictions_glob:
    print(prediction_glob.est)


Estimating biases using als...
RMSE: 0.8720
RMSE: 0.872022524720315
3.473602883115792
4.680413458798884
3.243962329835236
2.9400521821250774
3.4900036550826132
2.796605987522102
3.4807997732308382
3.298058699602936
3.8841204364826676
3.419708983190027
2.4810886733953175
3.9387028560125508
4.055451878016916
3.8357623504426357
3.222807894670891
3.0314290073765844
3.34520160844169
3.761816777432647
4.325863676427658
3.9501168648755853
3.7085916286894682
3.778266862433034
4.35055529740077
3.9240726678693787
4.165447447619005
3.0484991375121058
3.965072058476088
3.8076555531908003
4.0407317583905655
3.5787185059545696
3.3224985492081096
3.7953442494536485
3.44416547165432
3.3627650779377434
4.170664952353239
3.7506529228625176
4.190496759422332
3.0779507231266385
3.382192412975067
3.4544915719937546
2.721382730820488
3.920715756731352
3.913960234909997
3.795136432002734
3.6313828893891134
3.1403174813651837
3.4852828341163797
2.9900936239189337
3.1401110601968516
3.580595328711298
3.3054337

In [28]:
uid = [p.uid for p in predictions_glob]
iid = [p.iid for p in predictions_glob]
est = [p.est for p in predictions_glob]
predictions_df = pd.DataFrame({'userId': uid, 'movieId': iid, 'rating': est})
predictions_df

,userId,movieId,rating
0,484,1717,3.473603
1,1,457,4.680413
2,425,2001,3.243962
3,600,2506,2.940052
4,453,4310,3.490004
...,...,...,...
19288,372,1608,3.215823
19289,380,5959,3.630238
19290,122,3175,4.443035
19291,187,1673,3.987199


# Constructing model and predicting ratings

In [31]:
# Model construction
avg_rating = ratings_train[['movieId', 'rating']].groupby(by='movieId').mean()
	    
# Prediction
def predict_rating(df):
    # Input: 
	# 	df = a dataframe with two columns: userId, movieId
	# Output:
	#   a dataframe with three columns: userId, movieId, rating
	
   # new_df=pd.merge(df,predictions_df,on=["userId","movieId"],how="left")
    return df.join(avg_rating, on='movieId')

In [32]:
# Prepare df for prediction
r = ratings_valid[['userId', 'movieId']]

# Predict ratings
ratings_pred = predict_rating(r)
ratings_pred

,userId,movieId,rating
0,4,45,3.366667
1,4,52,3.520000
2,4,58,4.062500
3,4,222,3.928571
4,4,247,3.975000
...,...,...,...
2349,561,139385,3.860000
2350,561,146656,3.916667
2351,561,149406,3.416667
2352,561,160438,2.916667


In [33]:
from sklearn.metrics import mean_squared_error

r_true = ratings_valid['rating'].to_numpy()
r_pred = ratings_pred['rating'].to_numpy()

rmse = mean_squared_error(r_true, r_pred, squared=False)
print(f"RMSE = {rmse:.4f}")

RMSE = 0.9171
